##### IMPORTING NECESSARY LIBRARIES #######

In [ ]:
import os 
import cv2 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sb

###### EXTRACTING FILE FROM THE DIRECTORY ######

In [2]:
Data_path1 = os.listdir('D:/IMARTICUS LEARNING/HACKATHON/COIN DATASET')
len(Data_path1)

3059

#### CREATING COIN-DATASET AND TARGET VARIABLE(Rs. 5,10,25,59,100 ) ######

In [4]:
Target = []
Coin_data = []
for x in range(0,len(Data_path1)):
    #### READING EACH COIN IMAGE FILE ####
    img = cv2.imread('D:/IMARTICUS LEARNING/HACKATHON/COIN DATASET/'+ Data_path1[x])
    x11 = Data_path1[x][:3]
    #### CONVERTING TO GRAYSACE #####
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)  
    #### FIND THE CIRCLE (COIN) IN THE IMAGE AS X,Y - CENTRE AND R-RADIUS #####
    circle = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,120,50,30,minRadius =5,maxRadius=100)
    #### RESHAPE THE (X,Y,R) TO CHECK WHETHER CIRCLE PRESENT OT NOT
    circles = circle.reshape(-1,1)
    #### IF CIRCLE PRESENT 
    if circles[0] != 0 and circles[1] !=0 and circles[2] !=0:
        #### CENTER(X,Y) OF THE CIRCLE
        x = circle[0][0][0]
        y = circle[0][0][1]
        #### RADIUS OF THE CIRCLE
        r = circle[0][0][2]
        a = int(y-r)
        b = int(y+r)
        c = int(x-r)
        d = int(x+r)
        #### CROP ONLY THE IMAGE THAT CONTAINS THE COIN
        img = img[a:b,c:d]
        if img.shape[0] !=0  and img.shape[1] !=0:
            img = cv2.resize(img,(50,50))
            ##### STANDARDIZATION OF ARRAY   ####
            img = np.array(img)/255
            ##### CONVERT 2D TO 1D #####
            img = img.reshape(1,-1)
            Coin_data.append(img)
            #### CREATING TARGET VARIABLE BY USING THE FILE NAME
            if x11 == '5_1':
                Target.append(5)
            elif x11 == '10_':
                Target.append(10)
            elif x11 == '25_':
                Target.append(25)
            elif x11 == '50_':
                Target.append(50)
            elif x11 == '100':
                Target.append(100)
print(Target)

[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,

#### TRAIN TEST SPLIT #####

In [5]:
from sklearn.model_selection import train_test_split

In [59]:
xtrain,xtest,ytrain,ytest = train_test_split(Coin_data,Target,test_size=0.2,random_state=9)

#### CONVERTING THE FEATURE INTO 2D ######

In [60]:
np.array(xtrain).reshape(608,2500).shape, np.array(xtest).reshape(153,2500).shape

((608, 2500), (153, 2500))

In [61]:
np.array(ytrain).shape

(608,)

#### FITTING KNN MODEL ####

In [62]:
from sklearn.neighbors import KNeighborsClassifier 

In [63]:
KNN = KNeighborsClassifier(n_neighbors=13)
KNNfit = KNN.fit(np.array(xtrain).reshape(608,2500),np.array(ytrain))

In [64]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [65]:
confusion_matrix(ytest,KNNfit.predict(np.array(xtest).reshape(153,2500)))

array([[ 1,  0,  9,  0,  1],
       [ 0,  0,  1,  0,  0],
       [ 0,  0, 22,  0,  7],
       [ 0,  0,  5,  0,  7],
       [ 0,  0, 13,  0, 87]], dtype=int64)

In [66]:
accuracy_score(ytest,KNNfit.predict(np.array(xtest).reshape(153,2500)))

0.7189542483660131

#### MODEL FINE TUNING ######

In [67]:
#### FINE TUNING BY CHANGING RANDOM STATE #####
for x in range(1,30):
    xtrain,xtest,ytrain,ytest = train_test_split(Coin_data,Target,test_size=0.2,random_state=x)
    KNN = KNeighborsClassifier(n_neighbors=x)
    KNNfit = KNN.fit(np.array(xtrain).reshape(608,2500),np.array(ytrain))
    print(accuracy_score(ytest,KNNfit.predict(np.array(xtest).reshape(153,2500))))

0.6339869281045751
0.5359477124183006
0.6862745098039216
0.6339869281045751
0.6013071895424836
0.6274509803921569
0.6013071895424836
0.6535947712418301
0.7320261437908496
0.7058823529411765
0.673202614379085
0.6993464052287581
0.6535947712418301
0.7058823529411765
0.5555555555555556
0.6666666666666666
0.6666666666666666
0.6862745098039216
0.6993464052287581
0.673202614379085
0.6470588235294118
0.6535947712418301
0.6862745098039216
0.6470588235294118
0.6143790849673203
0.6274509803921569
0.738562091503268
0.6928104575163399
0.5816993464052288


In [57]:
##### FINE TUNING BY CHANGING THE NEIGHBOUR VALUE #####
for x in range(1,20):
    KNN = KNeighborsClassifier(n_neighbors=x)
    KNNfit = KNN.fit(np.array(xtrain).reshape(608,2500),np.array(ytrain))
    print(accuracy_score(ytest,KNNfit.predict(np.array(xtest).reshape(153,2500))))

0.6470588235294118
0.5555555555555556
0.6601307189542484
0.6339869281045751
0.7058823529411765
0.6797385620915033
0.6993464052287581
0.7189542483660131
0.7320261437908496
0.7254901960784313
0.7189542483660131
0.7058823529411765
0.7189542483660131
0.6862745098039216
0.6928104575163399
0.6862745098039216
0.7124183006535948
0.6928104575163399
0.7058823529411765


# THUS KNN MODEL IS FOUND TO BE BETTER IN PREDICTING THE COIN WITH AN ACCURACY SCORE OF 72 PER CENT